In [1]:
import boto3

# Inference variables
bucket_name = 'audiosnippet-bucket'

# AWS CLI arguments
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.Session().region_name
ecr_repo = 'monotone-inference'
processing_repo_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{ecr_repo}:latest'

print('region:', region)
print('processing_repo:', processing_repo_uri)

region: ap-southeast-1
processing_repo: 183640780274.dkr.ecr.ap-southeast-1.amazonaws.com/monotone-inference:latest


In [2]:
# # Dockerfile for cloud v1
# FROM nvcr.io/nvidia/tensorrt:21.03-py3

# RUN apt-get update && apt install -y \
#     nvidia-utils-450 \
#     ubuntu-drivers-common

# RUN apt update && \
#     apt-get install -y \
#     libsndfile1 \ 
#     libpq-dev gcc

# COPY requirements.txt requirements.txt

# RUN pip3 install -r requirements.txt

# LABEL maintainer="Xaltius"

# ENV PYTHONUNBUFFERED=TRUE

# ENV TOKENIZERS_PARALLELISM=false

# RUN mkdir -p /opt/ml/processing/inputs
# RUN mkdir -p /opt/ml/processing/models
# RUN mkdir -p /opt/ml/processing/logs
# RUN mkdir -p /opt/ml/processing/snippets

# COPY . /usr/src/app

In [3]:
# numpy==1.19.5
# tensorflow
# soundfile
# librosa
# boto3
# psycopg2-binary

In [4]:
# Create ECR repo and push docker image
!docker build --no-cache -t $processing_repo_uri docker
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
# !aws ecr create-repository --repository-name $ecr_repo
!docker push $processing_repo_uri
!docker image rm $processing_repo_uri

#1 [internal] load build definition from Dockerfile
#1 sha256:cc721352fc9fd8d56be74024c888a2a2b029e545e847265e5fe8211af9c0d460
#1 transferring dockerfile: 32B done
#1 DONE 0.0s

#2 [internal] load .dockerignore
#2 sha256:53113f56a08cb7c0df7a501f340590353f47704a79bfefcc8fac01ae7208282a
#2 transferring context: 2B done
#2 DONE 0.0s

#3 [internal] load metadata for nvcr.io/nvidia/tensorrt:21.03-py3
#3 sha256:3729ceaf29a15ab6082c0d2d92253ae6527a8b432b1a56b1e902dc4c2ee6d944
#3 DONE 2.5s

#4 [ 1/10] FROM nvcr.io/nvidia/tensorrt:21.03-py3@sha256:4f74d5a28b5fe9c900e258ae06c46be41b4e59dc67efe297e0a0dd087e094f3e
#4 sha256:b92171502b4dbccb95675c98097b2e28b18e18dbc9eb35e43096c3a3c0488f63
#4 CACHED

#7 [internal] load build context
#7 sha256:548ecbbed6ab24f8150ba2af8327ed23fdb6c51f9dab56e227b6d43bae249a50
#7 transferring context: 99B done
#7 DONE 0.0s

#5 [ 2/10] RUN apt-get update && apt install -y     nvidia-utils-450     ubuntu-drivers-common
#5 sha256:68a2e7998e0106906ae7a57375e43d9491abfcce9b5

Login Succeeded



Logging in with your password grants your terminal complete access to your account. 
For better security, log in with a limited-privilege personal access token. Learn more at https://docs.docker.com/go/access-tokens/


The push refers to repository [183640780274.dkr.ecr.ap-southeast-1.amazonaws.com/monotone-inference]
59483a6a0fce: Preparing
5d399a148bfd: Preparing
2eda4c16a41b: Preparing
bf30b7e47c5d: Preparing
364b3cf56ea7: Preparing
9b1ebb84ccf4: Preparing
d49474536f60: Preparing
1983da5ada05: Preparing
b25b20555ccf: Preparing
2dc9036eb10f: Preparing
18177e156f9e: Preparing
6f76a923f98a: Preparing
8f7d3a80f011: Preparing
743c2ba5e1ba: Preparing
5e439cd64380: Preparing
7880124e4116: Preparing
ff487f9b5dda: Preparing
a23d73b040c6: Preparing
41646a22b454: Preparing
47ccf219c15c: Preparing
01cb3db8bf95: Preparing
0b45ac6c02ae: Preparing
e13a19a660a0: Preparing
fef2f834ce09: Preparing
a8a04e09846a: Preparing
914c9a636493: Preparing
cb66cb12cb15: Preparing
5f70bf18a086: Preparing
5a46c3025156: Preparing
43f4c95269e9: Preparing
d95172d4c20a: Preparing
91e45585ed5f: Preparing
5176c7399af5: Preparing
b51dc3022331: Preparing
65a104f315ea: Preparing
df2ab710b65a: Preparing
e77bb70c5efa: Preparing
f6504a8708c